In [1]:
!pip install pandas
!pip install optuna
!pip install optuna-dashboard
!pip installl scikit
!pip install imbalanced-learn

ERROR: unknown command "installl" - maybe you meant "install"


In [2]:
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)
from sklearn.metrics import recall_score, accuracy_score,f1_score, precision_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline
import warnings
import optuna
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
random_state = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
raw_dataset = pd.read_csv("./data/processed_data.csv") #data has X and Y
X = raw_dataset.drop(columns=["DR"])
Y = pd.DataFrame(raw_dataset["DR"])

#* 90/10 split for training and final test
X_FOR_FOLDS, X_FINAL_TEST, Y_FOR_FOLDS, Y_FINAL_TEST = train_test_split(X, Y, test_size=0.1, random_state=random_state, stratify=Y)

Using device: cuda


Helper functions that don't need tweakin

In [4]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from modularModels1 import BlockMaker, modularNN, BasicModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)

def init_weights(model): #tested already
    if isinstance(model, nn.Linear):  # Apply only to linear layers
        nn.init.xavier_uniform_(model.weight)
        if model.bias is not None:
            nn.init.zeros_(model.bias)
            
def fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64, device=device):
    train_dataset = TensorDataset(
        torch.tensor(train_x.values,dtype=torch.float32).to(device), 
        torch.tensor(train_y.values,dtype=torch.float32).to(device))
    val_dataset = TensorDataset(
        torch.tensor(test_x.values,dtype=torch.float32).to(device), 
        torch.tensor(test_y.values,dtype=torch.float32).to(device))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
    return train_loader, val_loader 

def get_feature_count(loader):
    """returns the number of features in the dataset"""
    return next(iter(loader))[0].shape[1]

Using cuda


In [5]:
from Criterion_Models import *
def criterion_mapping(criterion_choice:str, pos_weight:float=None, alpha:float=None, gamma:float=None):
    """
    Feel free to add any custom loss functions here.
    returns function for criterion
    """
    if criterion_choice == "FocalLoss":
        return FocalLoss(alpha =alpha, gamma=gamma)
    elif criterion_choice == "DiceLoss":
        return DiceLoss()
    elif criterion_choice == "BCEWithLogitsLoss":
        return nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight])) if pos_weight else nn.BCEWithLogitsLoss()
    return nn.BCEWithLogitsLoss() 

Helper functions that could use tweakin to improve model performance

In [6]:
def FOLDS_GENERATOR(X, Y, normalisation_method=MinMaxScaler(), n_splits=5, random_state=None, oversampler=None):
    """
    Generates stratified folds with specified normalization.
    
    For list of scalers, see:
    https://scikit-learn.org/stable/api/sklearn.preprocessing.html
    
    For more details on scaling and normalization effects, see:
    https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#
    
    normalisation_method should be an instance of a scaler, e.g.,
    - MinMaxScaler()
    - MaxAbsScaler()
    - Quantile_Transform(output_distribution='uniform')
    
    Returns a list of tuples, each containing:
    (X_train_scaled, X_test_scaled, Y_train, Y_test), representing data for each fold
    """
    kF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    kFolds_list = []
    
    for fold, (train_idx, test_idx) in enumerate(kF.split(X, Y)):
        # Split the data into training and testing sets for this fold
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
        
        # IsolationForest for outlier removal (optional)
        iso_forest = IsolationForest(contamination=0.05, random_state=random_state)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UserWarning)
            outliers = iso_forest.fit_predict(X_train)    
        X_train = X_train[outliers == 1]
        Y_train = Y_train[outliers == 1]
        
        # Scale the entire data (binary and continuous together)
        X_train_scaled = normalisation_method.fit_transform(X_train)
        X_test_scaled = normalisation_method.transform(X_test)

        # Handle oversampling if needed
        if oversampler:
            # Apply oversampling to both features and target
            X_train_scaled, Y_train = oversampler.fit_resample(X_train_scaled, Y_train)

        # Convert scaled data back to DataFrame with the correct column names
        X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
        X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
        
        # Handle community columns
        community_cols = [col for col in X_train_scaled.columns if col.startswith('Community')] 

        # Check for rows where multiple communities are flagged
        for idx, row in X_train_scaled[community_cols].iterrows():
            if set(np.unique(row)) != {0, 1}:  # If the unique values aren't just 0 or 1
                # Fix row by ensuring only one community is marked
                X_train_scaled.loc[idx, community_cols] = 0  # Set all community columns to 0
                max_col = row.idxmax()  # Find the column with the maximum value
                X_train_scaled.at[idx, max_col] = 1  # Set the column with the max value to 1

        # Ensure 'gender' is still binary (0 or 1)
        if 'Gender' in X_train_scaled.columns:
            X_train_scaled['Gender'] = (X_train_scaled['Gender'] > 0.5).astype(int)
            X_test_scaled['Gender'] = (X_test_scaled['Gender'] > 0.5).astype(int)
        
        # Append the processed fold to the list
        kFolds_list.append((X_train_scaled, X_test_scaled, Y_train, Y_test))
        
        print(f"Fold: {fold+1}, Train: {kFolds_list[fold][0].shape}, Test: {kFolds_list[fold][1].shape}")
    
    return kFolds_list


In [7]:
def train_and_evaluate(model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=20, patience=5, device=device):
    if isinstance(model.last_layer(), nn.Sigmoid) and isinstance(criterion, nn.BCEWithLogitsLoss):
        raise ValueError("Model output is Sigmoid but criterion is BCEWithLogitsLoss. Please check your model and criterion compatibility.")
    best_val_loss = float('inf')
    best_model_state = None
    wait = 0

    
    #* Epoch Training loop for this fold
    for epoch in range(1,epochs+1):
        #* Set model to training mode: essential for dropout and batch norm layers
        model.train()
        running_loss = 0.0 #? loss for this epoch
        #* Mini-batch training loop
        for batch, (inputs, labels) in enumerate(train_loader,start=1):
            optimiser.zero_grad() #? Zero the gradients
            outputs = model(inputs) #? Forward pass through the model
            loss = criterion(outputs, labels) #? Calculate loss
            loss.backward() #? Backpropagation
            running_loss += loss.item()
            optimiser.step() #? Update weights
            if isinstance(scheduler,torch.optim.lr_scheduler.OneCycleLR):
                scheduler.step()
        if not isinstance(scheduler,torch.optim.lr_scheduler.OneCycleLR):
            scheduler.step()
                
        train_loss = running_loss / len(train_loader)
        # print(f"Epoch: {epoch}, training loss: {train_loss:.4f}")
    
        #* Now we evaluate the model on the validation set, to track training vs validation loss
        model.eval() #? Set model to evaluation mode
        with torch.no_grad(): #? No need to track gradients during evaluation
            val_loss = 0.0    
            for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                outputs = model(inputs)
                # labels = labels.cpu() 
                loss = criterion(outputs, labels)
                val_loss += loss.item() #? Calculate loss
            avg_val_loss = val_loss / len(val_loader)
            # print(f"Epoch: {epoch}, training loss: {train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
            # print(f"Epoch: {epoch}".ljust(12), f"training loss:{train_loss:.4f}".ljust(12), f"Val Loss: {avg_val_loss:.4f}",end="\r")
        
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            wait = 0
        elif avg_val_loss*0.85 <= best_val_loss:
                wait = 0
        else:
            wait += 1
        if wait >= patience:
            print(f"Early stopping triggered at epoch {epoch}, best val loss: {best_val_loss:.4f}")
            break
        print(f"Epoch: {epoch}".ljust(12), f"training loss:{train_loss:.4f}".ljust(12), f"best_val_loss:{best_val_loss:.4f}".ljust(12), f"Val Loss: {avg_val_loss:.4f}",end="\r")
    #* Use best model to calculate metrics on the validation set
    #! must be outside epoch loop, it comes after the training and cv loop
    model.load_state_dict(best_model_state) #? Load the best model state
    with torch.no_grad():
        for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                labels = labels.cpu() 
                predictions = (torch.sigmoid(outputs) < 0.5).float().cpu().numpy()
                
                val_loss += loss.item() #? Calculate loss
                
    #! The following should have length equal to fold number           
    accuracy=accuracy_score(labels, predictions) 
    precision=precision_score(labels, predictions, pos_label=1, zero_division=0)
    recall=recall_score(labels, predictions, pos_label=1)
    f1=f1_score(labels, predictions, pos_label=1)
    auc=roc_auc_score(labels, predictions)
    
    return model, accuracy, precision, recall, f1, auc


In [8]:


class BinaryClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1),
            # nn.Sigmoid()
   
        )

    def forward(self, x):
        return self.net(x)
    
    def last_layer(self):
        return self.net[-1]

In [9]:
import torch
import torch.nn as nn

class FeedForwardBlock(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.1, activation=nn.ReLU):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.BatchNorm1d(out_features),
            activation(),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.block(x)


class MyModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout=0.2):
        super().__init__()
        
        # A couple of FeedForward blocks
        self.block1 = FeedForwardBlock(input_dim, hidden_dim, dropout)
        self.block2 = FeedForwardBlock(hidden_dim, hidden_dim, dropout)
        self.block3 = FeedForwardBlock(hidden_dim, output_dim, dropout)

        # Final output layer (could be softmax, sigmoid, or whatever your target is)
        self.output_layer = nn.Linear(output_dim, 1)  # Just in case you're doing regression or binary classification

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.output_layer(x)  # Final linear layer
        return x
    
    def last_layer(self):
        return self.output_layer


In [10]:
def maximise_combined_score(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    epochs = 10000
    random_state = 42
    oversampler = ADASYN(sampling_strategy='minority', n_neighbors=5, random_state=random_state)
    if isinstance(oversampler, ADASYN):
        n_neighbours = trial.suggest_int("n_neighbours", 1, 10)
        oversampler = ADASYN(sampling_strategy='minority', n_neighbors=n_neighbours, random_state=random_state)
    
    normalisation_method = trial.suggest_categorical("normalisation_method", [
        "MinMaxScaler",
        # "MaxAbsScaler",
        # "StandardScaler",
        # "RobustScaler",
        # "PowerTransformer",
        # "QuantileTransformer",
    ])
    if normalisation_method:
        if normalisation_method == "MinMaxScaler":
            normalisation_method = MinMaxScaler()
        elif normalisation_method == "MaxAbsScaler":
            normalisation_method = MaxAbsScaler()
        elif normalisation_method == "StandardScaler":
            normalisation_method = StandardScaler()
        elif normalisation_method == "RobustScaler":
            normalisation_method = RobustScaler()
        elif normalisation_method == "PowerTransformer":
            normalisation_method = PowerTransformer()
        elif normalisation_method == "QuantileTransformer":
            normalisation_method = QuantileTransformer(output_distribution='uniform')
        else:
            normalisation_method = MinMaxScaler()
    
    kFolds = FOLDS_GENERATOR(X_FOR_FOLDS, Y_FOR_FOLDS, 
                         normalisation_method = normalisation_method, 
                         n_splits=5, 
                         oversampler = oversampler, random_state=42)
                        
    # Model hyperparameters (first-level optimization)
    hidden_dim = trial.suggest_int("hidden_dim", 28, 256)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    initial_lr = trial.suggest_float("initial_lr", 1e-5, 1e-3, log=True)
    max_lr = trial.suggest_float("max_lr", 1e-3, 1e-1, log=True)
    
    # Loss function hyperparameters
    criterion_choice = trial.suggest_categorical("criterion", ["BCEWithLogitsLoss", "FocalLoss"]) 
    
    # Hyperparameter exploration optimization
    if criterion_choice == "BCEWithLogitsLoss":
        pos_weight = trial.suggest_int("pos_weight", 1, 10)
        alpha = None
        gamma = None
    elif criterion_choice == "FocalLoss":
        pos_weight= None
        alpha = trial.suggest_float("alpha", 1, 10)
        gamma = trial.suggest_float("gamma", 0, 5)
    else:
        pos_weight = None
    
    # Initialize lists for metrics across folds
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    auc_list = []

    # Cross-validation loop
    for fold, (train_x, test_x, train_y, test_y) in enumerate(kFolds, start=1):
        # Create DataLoader for current fold
        train_loader, val_loader = fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=512, device=device)
        # Calculate steps_per_epoch from the current fold's train_loader
        train_loader_len = len(train_loader)
        
        # Instantiate and initialize the model
        # model = BinaryClassifier(input_dim=get_feature_count(train_loader), hidden_dim=hidden_dim, dropout=dropout)
        model = MyModel(input_dim=get_feature_count(train_loader), hidden_dim=hidden_dim, output_dim=hidden_dim, dropout=dropout)
        model.to(device)
        model.apply(init_weights)
        
        # Map the choice to the actual loss function
        criterion = criterion_mapping(criterion_choice, pos_weight, alpha, gamma).to(device)
        optimiser = optim.Adam(model.parameters(), lr=initial_lr)

        
        # Initialize scheduler
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimiser,
            max_lr=max_lr,
            steps_per_epoch=train_loader_len,
            epochs=epochs,
            anneal_strategy='linear'
        )
        print(f"Fold {fold}:")
        # Train and evaluate the model on the current fold
        model, accuracy, precision, recall, f1, auc = train_and_evaluate(
            model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=epochs, patience=30, device=device
        )
        print(f"Accuracy: {accuracy:.4f}, precision: {precision:.4f}, recall: {recall:.4f}, f1: {f1:.4f}, auc: {auc:.4f}")

        # Append the metrics from the current fold
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
        auc_list.append(auc)

    # Calculate the average metrics across all folds
    avg_accuracy = np.sum(accuracy_list) / len(accuracy_list)
    avg_precision = np.sum(precision_list) / len(precision_list)
    avg_recall = np.sum(recall_list) / len(recall_list)
    avg_f1 = np.sum(f1_list) / len(f1_list)
    avg_auc = np.sum(auc_list) / len(auc_list)

    # Combine metrics into a single "score"
    # combined_score = (avg_f1 + avg_precision + avg_recall + avg_accuracy + avg_auc) / 5
    combined_score = avg_f1

    return combined_score


In [11]:
import threading
import optuna
from optuna_dashboard import run_server
!fuser -k 8080/tcp

# Define your persistent storage
storage = "sqlite:///optuna_study3.db"

# Create or load your study
study_name = "my_cursed_study"
try:
    study = optuna.load_study(study_name=study_name, storage=storage)
except KeyError:
    study = optuna.create_study(study_name=study_name, direction="maximize", storage=storage)

# Start Optuna Dashboard in a separate thread
dashboard_thread = threading.Thread(target=lambda: run_server(storage), daemon=True)
dashboard_thread.start()

# Run optimization
study.optimize(maximise_combined_score, n_trials=30)

# Print results
print("Best trial:")
trial = study.best_trial
print(f"  Combined score: {trial.value}")
print("  Best hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-14 04:41:16,033] A new study created in RDB with name: my_cursed_study
Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.



Using device: cuda
Fold: 1, Train: (7720, 28), Test: (1149, 28)
Fold: 2, Train: (7933, 28), Test: (1149, 28)
Fold: 3, Train: (7844, 28), Test: (1148, 28)
Fold: 4, Train: (7974, 28), Test: (1148, 28)
Fold: 5, Train: (7984, 28), Test: (1148, 28)
Fold 1:
Early stopping triggered at epoch 365, best val loss: 0.8345oss: 1.0479
Accuracy: 0.2019, precision: 0.0884, recall: 0.7414, f1: 0.1579, auc: 0.4414
Fold 2:
Early stopping triggered at epoch 33, best val loss: 0.7077Loss: 0.9108
Accuracy: 0.4970, precision: 0.0266, recall: 0.1121, f1: 0.0430, auc: 0.3261
Fold 3:
Early stopping triggered at epoch 34, best val loss: 0.6658Loss: 0.9454
Accuracy: 0.5375, precision: 0.0419, recall: 0.1638, f1: 0.0668, auc: 0.3716
Fold 4:
Early stopping triggered at epoch 231, best val loss: 0.8836oss: 1.1072
Accuracy: 0.2082, precision: 0.0857, recall: 0.7069, f1: 0.1528, auc: 0.4295
Fold 5:


[I 2025-04-14 04:42:43,008] Trial 0 finished with value: 0.1129849393384675 and parameters: {'n_neighbours': 1, 'normalisation_method': 'MinMaxScaler', 'hidden_dim': 144, 'dropout': 0.13962726108715434, 'initial_lr': 0.0004309950799024789, 'max_lr': 0.0024107424878657287, 'criterion': 'BCEWithLogitsLoss', 'pos_weight': 2}. Best is trial 0 with value: 0.1129849393384675.


Early stopping triggered at epoch 187, best val loss: 0.8213
Accuracy: 0.2666, precision: 0.0818, recall: 0.6121, f1: 0.1443, auc: 0.4199
Using device: cuda
Fold: 1, Train: (8038, 28), Test: (1149, 28)
Fold: 2, Train: (7863, 28), Test: (1149, 28)
Fold: 3, Train: (7869, 28), Test: (1148, 28)
Fold: 4, Train: (7929, 28), Test: (1148, 28)
Fold: 5, Train: (7974, 28), Test: (1148, 28)
Fold 1:
Early stopping triggered at epoch 33, best val loss: 0.4762Loss: 0.6732
Accuracy: 0.5100, precision: 0.0521, recall: 0.2241, f1: 0.0846, auc: 0.3831
Fold 2:
Early stopping triggered at epoch 35, best val loss: 0.4257Loss: 0.7595
Accuracy: 0.5083, precision: 0.0465, recall: 0.1983, f1: 0.0753, auc: 0.3707
Fold 3:
Early stopping triggered at epoch 362, best val loss: 0.5603oss: 0.6958
Accuracy: 0.2866, precision: 0.0698, recall: 0.4914, f1: 0.1222, auc: 0.3775
Fold 4:
Early stopping triggered at epoch 344, best val loss: 0.5937oss: 0.7422
Accuracy: 0.2953, precision: 0.0738, recall: 0.5172, f1: 0.1292, au

[I 2025-04-14 04:44:41,287] Trial 1 finished with value: 0.11242004051220982 and parameters: {'n_neighbours': 2, 'normalisation_method': 'MinMaxScaler', 'hidden_dim': 155, 'dropout': 0.2516359405709947, 'initial_lr': 0.00022817518077666336, 'max_lr': 0.0015430427069904776, 'criterion': 'FocalLoss', 'alpha': 9.760302607482629, 'gamma': 3.623053506022107}. Best is trial 0 with value: 0.1129849393384675.


...

Early stopping triggered at epoch 44, best val loss: 1.5077
Accuracy: 0.2596, precision: 0.0820, recall: 0.6207, f1: 0.1449, auc: 0.4198
Using device: cuda
Fold: 1, Train: (7996, 28), Test: (1149, 28)
Fold: 2, Train: (7899, 28), Test: (1149, 28)
Fold: 3, Train: (7902, 28), Test: (1148, 28)
Fold: 4, Train: (7943, 28), Test: (1148, 28)
Fold: 5, Train: (7959, 28), Test: (1148, 28)
Fold 1:
Early stopping triggered at epoch 52, best val loss: 1.7014Loss: 3.7282
Accuracy: 0.3211, precision: 0.0754, recall: 0.5086, f1: 0.1314, auc: 0.4044
Fold 2:
Early stopping triggered at epoch 53, best val loss: 1.6575Loss: 3.3699
Accuracy: 0.2620, precision: 0.0674, recall: 0.4914, f1: 0.1185, auc: 0.3638
Fold 3:
Early stopping triggered at epoch 40, best val loss: 1.8091Loss: 4.0491
Accuracy: 0.2544, precision: 0.0718, recall: 0.5345, f1: 0.1265, auc: 0.3787
Fold 4:
Early stopping triggered at epoch 41, best val loss: 1.6945Loss: 3.4600
Accuracy: 0.3101, precision: 0.0848, recall: 0.5948, f1: 0.1484, auc

[I 2025-04-14 05:03:59,435] Trial 28 finished with value: 0.1337258510593625 and parameters: {'n_neighbours': 10, 'normalisation_method': 'MinMaxScaler', 'hidden_dim': 186, 'dropout': 0.16287020580855993, 'initial_lr': 1.7530985619836235e-05, 'max_lr': 0.04495246417279645, 'criterion': 'BCEWithLogitsLoss', 'pos_weight': 9}. Best is trial 20 with value: 0.1495982976698472.


Early stopping triggered at epoch 38, best val loss: 1.5768
Accuracy: 0.3258, precision: 0.0825, recall: 0.5603, f1: 0.1438, auc: 0.4299
Using device: cuda
Fold: 1, Train: (7910, 28), Test: (1149, 28)
Fold: 2, Train: (7926, 28), Test: (1149, 28)
Fold: 3, Train: (7964, 28), Test: (1148, 28)
Fold: 4, Train: (7982, 28), Test: (1148, 28)
Fold: 5, Train: (7889, 28), Test: (1148, 28)
Fold 1:
Early stopping triggered at epoch 31, best val loss: 1.4642Loss: 2.3446
Accuracy: 0.3586, precision: 0.0682, recall: 0.4224, f1: 0.1174, auc: 0.3869
Fold 2:
Early stopping triggered at epoch 38, best val loss: 1.5198Loss: 2.5624
Accuracy: 0.3238, precision: 0.0680, recall: 0.4483, f1: 0.1180, auc: 0.3790
Fold 3:
Early stopping triggered at epoch 42, best val loss: 1.8439Loss: 3.8868
Accuracy: 0.2726, precision: 0.0674, recall: 0.4828, f1: 0.1183, auc: 0.3659
Fold 4:
Early stopping triggered at epoch 31, best val loss: 1.3322Loss: 2.8804
Accuracy: 0.3641, precision: 0.0724, recall: 0.4483, f1: 0.1247, auc

[I 2025-04-14 05:04:19,252] Trial 29 finished with value: 0.11815993968773683 and parameters: {'n_neighbours': 7, 'normalisation_method': 'MinMaxScaler', 'hidden_dim': 238, 'dropout': 0.12949676651213293, 'initial_lr': 2.83434635779415e-05, 'max_lr': 0.021982953055555348, 'criterion': 'BCEWithLogitsLoss', 'pos_weight': 10}. Best is trial 20 with value: 0.1495982976698472.


Early stopping triggered at epoch 31, best val loss: 1.5540Loss: 2.9379
Accuracy: 0.4085, precision: 0.0663, recall: 0.3707, f1: 0.1124, auc: 0.3917
Best trial:
  Combined score: 0.1495982976698472
  Best hyperparameters:
    n_neighbours: 7
    normalisation_method: MinMaxScaler
    hidden_dim: 255
    dropout: 0.13453206643934615
    initial_lr: 3.405906311970348e-05
    max_lr: 0.05323481728278594
    criterion: BCEWithLogitsLoss
    pos_weight: 8
